# Notebook to process output from thecontentmine

This notebook illustrates the basic process of processing each resource in each paper and allocates a score.

In [1]:
import pandas
import json

import process_urls

In [2]:
with open('amw_token', 'r') as f:
    github_token = f.read()

In [3]:
#Read JSON data into the datastore variable
with open('../sample_data/small_dict_of_papers.json', 'r') as f:
    dict_of_papers = json.load(f)

In [4]:
resources_list = process_urls.process_papers_dict(dict_of_papers, verbose=True, 
                                                  github_token=github_token)

processing paper with doi 10.1038/s41467-018-03297-7
check licence for URL https://github.com/YeatmanLab/AFQ-Browser_data
URL https://github.com/YeatmanLab/AFQ-Browser_data does not have a licence
check licence for URL https://github.com/YeatmanLab/AFQ-Browser_data/blob/master/AFQ-Browser_ALSexample/Reproducing-Sarica2017-Figure3.ipynb
skipping licence check for URL https://github.com/YeatmanLab/AFQ-Browser_data/blob/master/AFQ-Browser_ALSexample/Reproducing-Sarica2017-Figure3.ipynb
check licence for URL https://github.com/YeatmanLab/AFQ-Browser_data/blob/master/AFQ-Browser_ALSexample/Figure6.ipynb
skipping licence check for URL https://github.com/YeatmanLab/AFQ-Browser_data/blob/master/AFQ-Browser_ALSexample/Figure6.ipynb
check licence for URL https://github.com
URL https://github.com does not have a licence
check licence for URL https://github.com/YeatmanLab/AFQ-Browser
URL https://github.com/YeatmanLab/AFQ-Browser has a licence
check licence for URL https://github.com/YeatmanLab/AFQ

## Analysis of the data

We can store the data in a flat SQL database, but here we stuff it into a Pandas dataframe to allow some analysis.

In [5]:
url_df = pandas.DataFrame.from_dict(resources_list)

In [6]:
url_df

,doi,licence_exists,pub_date,resolves,resourcetype,score,timestamp,url
0,10.1038/s41467-018-03297-7,False,2018-03-01,True,github,1,2018-03-28T16:00:58.504635,https://github.com/YeatmanLab/AFQ-Browser_data
1,10.1038/s41467-018-03297-7,None,2018-03-01,True,github,1,2018-03-28T16:00:59.981803,https://github.com/YeatmanLab/AFQ-Browser_data...
2,10.1038/s41467-018-03297-7,None,2018-03-01,True,github,1,2018-03-28T16:01:00.838839,https://github.com/YeatmanLab/AFQ-Browser_data...
3,10.1038/s41467-018-03297-7,False,2018-03-01,True,github,1,2018-03-28T16:01:01.492537,https://github.com
4,10.1038/s41467-018-03297-7,True,2018-03-01,True,github,2,2018-03-28T16:01:01.998207,https://github.com/YeatmanLab/AFQ-Browser
5,10.1038/s41467-018-03297-7,False,2018-03-01,True,github,1,2018-03-28T16:01:03.196994,https://github.com/YeatmanLab/AFQ-Browser/
6,10.1038/s41467-018-03297-7,False,2018-03-01,True,github,1,2018-03-28T16:01:04.613655,https://github.com/yeatmanlab/AFQ-Browser_data
7,10.1038/s41467-018-03297-7,False,2018-03-01,True,github,1,2018-03-28T16:01:05.785123,https://github.com/yeatmanlab/AFQBrowser-demo/
8,10.1038/s41467-018-03297-7,False,2018-03-01,True,github,1,2018-03-28T16:01:07.583452,https://github.com/yeatmanlab/AFQ-Browser-MSex...
9,10.1038/s41467-018-03297-7,False,2018-03-01,True,github,1,2018-03-28T16:01:09.295704,https://github.com/yeatmanlab/Sarica_2017
